In [1]:
import tabula
import pandas as pd
import re
from json import load, dump

In [2]:
# Convert neat pdf to neater dataframe
raw_df = pd.DataFrame(tabula.read_pdf("./raw_data/deanlist_2017_fall_mainland.pdf", pages="all"))
raw_df.columns = ["name", "city", "country", "school", "list"]
raw_df.head()

,name,city,country,school,list
0,Avery Ao,Taiyuan,China,Colege of Agricultural and Life Sciences,Dean's List
1,Dongrui Bai,Xian,China,Colege of Letters & Science,Dean's List
2,Tony Bai,Beijing 100062,China,Colege of Engineering,Dean's Honor List
3,Yuchen Bai,Beijing,China,Colege of Letters & Science,Dean's List
4,Xinya Bi,Jinan City Shandong,China,School of Business,Dean's List


In [3]:
raw_df_hk = pd.DataFrame(tabula.read_pdf("./raw_data/deanlist_2017_fall_hongkong.pdf", pages="all"))
raw_df_hk.columns = ["name", "city", "country", "school", "list"]
raw_df_hk.head()

,name,city,country,school,list
0,Christie Cheng,Hong Kong,Hong Kong,Colege of Agricultural and Life Sciences,Dean's List
1,Meggie Cook,Clearwater Bay Kowloon,Hong Kong,Colege of Letters & Science,Dean's List
2,Christie Leung,Hong Kong,Hong Kong,Colege of Agricultural and Life Sciences,Dean's List
3,Tess Lo,Hong Kong,Hong Kong,Colege of Letters & Science,Dean's List
4,Yi Luo,KOWLOON,Hong Kong,School of Human Ecology,Dean's Honor List


In [4]:
# Need to specially handle taiwan data, tabula has some issue
raw_df_tw = pd.DataFrame(tabula.read_pdf("./raw_data/deanlist_2017_fall_taiwan.pdf", pages="all"))
raw_df_tw.columns = ["name", "city", "country", "list"]
for index, row in raw_df_tw.iterrows():
    raw_df_tw.loc[index, "school"] = re.search('China\s(.+)', row["country"]).groups()[0]
    raw_df_tw.loc[index, "country"] = "Taiwan"
raw_df_tw.head()

,name,city,country,list,school
0,Wei-Ting Chen,Taoyuan City,Taiwan,Dean's List,Colege of Letters & Science
1,Tina Cheng,Taipei City,Taiwan,High Honor Rol,School of Pharmacy
2,Ting-Chia Kan,Taoyuan,Taiwan,Dean's List,Colege of Letters & Science
3,Justine Kao,Kaohsiung,Taiwan,Dean's List,Colege of Agricultural and Life Sciences
4,Amy Lin,Taichung,Taiwan,Dean's List,Colege of Agricultural and Life Sciences


In [5]:
# Reserve an original copy for reference
df = raw_df.copy()
df_tw = raw_df_tw.copy()
df_hk = raw_df_hk.copy()

# Clean the dataframe
df["city"] = df["city"].apply(lambda city: re.split('\W+', city.lower())[0])
df_tw["city"] = df_tw["city"].apply(lambda city: re.split('\W+', city.lower())[0])
df_hk["city"] = "hongkong"

# Further clean taiwan city names
df_tw.loc[df_tw["city"] == "new", "city"] = "newtaipei"
df_tw.head()

,name,city,country,list,school
0,Wei-Ting Chen,taoyuan,Taiwan,Dean's List,Colege of Letters & Science
1,Tina Cheng,taipei,Taiwan,High Honor Rol,School of Pharmacy
2,Ting-Chia Kan,taoyuan,Taiwan,Dean's List,Colege of Letters & Science
3,Justine Kao,kaohsiung,Taiwan,Dean's List,Colege of Agricultural and Life Sciences
4,Amy Lin,taichung,Taiwan,Dean's List,Colege of Agricultural and Life Sciences


In [6]:
# Concatenate three dataframes
frames = [df, df_hk, df_tw]
tmp = pd.concat(frames, ignore_index=True)
df = tmp[["name", "city", "country", "school", "list"]]
df.shape

(685, 5)

In [7]:
print(len(set(df["city"])))
set(df["city"])

112


{'anqing',
 'anshan',
 'baoding',
 'beijing',
 'beijng',
 'binzhou',
 'changchun',
 'changde',
 'changsha',
 'changshu',
 'changzhou',
 'chengdu',
 'chongqing',
 'dalian',
 'daqing',
 'dongiyng',
 'dunhuang',
 'foshan',
 'fuding',
 'fuzhou',
 'gaoyou',
 'guangzhou',
 'gui',
 'guilin',
 'guiyang',
 'haikou',
 'hangzhou',
 'harbin',
 'hefei',
 'hohhot',
 'hongkong',
 'huizhou',
 'jianyang',
 'jiaxing',
 'jinan',
 'jinhua',
 'jining',
 'jinzhou',
 'kaifeng',
 'kaohsiung',
 'karamay',
 'kunming',
 'kunshan',
 'langfang',
 'lanzhou',
 'leshan',
 'liaoyang',
 'linyi',
 'loudi',
 'luoyang',
 'luzhou',
 'maanshan',
 'meishan',
 'mianyang',
 'nanchang',
 'nanjing',
 'nanning',
 'nantong',
 'nanyang',
 'newtaipei',
 'ningbo',
 'pudong',
 'qingdao',
 'quanzhou',
 'quzhou',
 'sh',
 'shanghai',
 'shantou',
 'shaoxing',
 'shengzhou',
 'shenyang',
 'shenzhen',
 'shijiazhuang',
 'shouguang',
 'suqian',
 'suzhou',
 'taian',
 'taichung',
 'tainan',
 'taipei',
 'taiyuan',
 'taiyuanshi',
 'tangshan',
 'ta

In [8]:
# Some city names are not still not cleaned, such as `sh`
# Those case are still minor, so we can fix it manually
city_coord = load(open("china_coord.json", "r"))
for c in df["city"]:
    if c not in city_coord:
        print(c)

beijng
pudong
xi
dongiyng
sh
xi
xi
xi
xi
zhongqing
yongtaizhuang
xi
gui
xi
xi
taiyuanshi
taiyuanshi


In [9]:
# Fix students' typo
pd.options.mode.chained_assignment = None 
df.drop(df.index[df['city'] == "sh"], inplace=True)
df.loc[df.index[df['city'] == "beijng"], 'city'] = "beijing"
df.loc[df.index[df['city'] == "xi"], 'city'] = "xi'an"
df.loc[df.index[df['city'] == "xian"], 'city'] = "xi'an"
df.loc[df.index[df['city'] == "pudong"], 'city'] = "shanghai"
df.loc[df.index[df['city'] == "dongiyng"], 'city'] = "dongying"
df.loc[df.index[df['city'] == "zhongqing"], 'city'] = "chongqing"
df.loc[df.index[df['city'] == "yongtaizhuang"], 'city'] = "beijing"
df.loc[df.index[df['city'] == "gui"], 'city'] = "guiyang"
df.loc[df.index[df['city'] == "taiyuanshi"], 'city'] = "taiyuan"

In [10]:
# Add latitude and longtitude info the dataframe
lat, long = [], []
for index, row in df.iterrows():
    df.loc[index, "lat"] = city_coord[row["city"]]["coord"][0]
    df.loc[index, "long"] = city_coord[row["city"]]["coord"][1]
df.head()

,name,city,country,school,list,lat,long
0,Avery Ao,taiyuan,China,Colege of Agricultural and Life Sciences,Dean's List,37.8903,112.5509
1,Dongrui Bai,xi'an,China,Colege of Letters & Science,Dean's List,34.2778,108.9531
2,Tony Bai,beijing,China,Colege of Engineering,Dean's Honor List,39.9300,116.3956
3,Yuchen Bai,beijing,China,Colege of Letters & Science,Dean's List,39.9300,116.3956
4,Xinya Bi,jinan,China,School of Business,Dean's List,36.6828,117.0250


In [11]:
# Write the dataframe to a csv
df.to_csv("deanlist_2017_fall.csv", index=False)